# Setup

In [1]:
from openai import OpenAI
from openai import embeddings

In [2]:
from dotenv import load_dotenv
import os
import json

load_dotenv()



True

In [3]:
print(os.getenv("OPENAI_API_KEY")[:20], "...")
print(os.getenv("OPENAI_ORG")[:8], "...")
print(os.getenv("OPENAI_PROJECT")[:8], "...")

sk-proj-eUanLfMUnbXn ...
org-8d28 ...
proj_30j ...


## Envinronment Variables

Create a .env File: In the root of your project directory, create a file named `.env`. This file will hold your environment variables in the form of key-value pairs. For example:

```ini
    OPENAI_API_KEY = <openai_apikey>


    # optional 
    OPENAI_ORG = <openai_org_id>
    OPENAI_PROJECT = <openai_project_id>

```

In [4]:

# Load environment variables from .env file
load_dotenv()

# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
# model = "gpt-5-mini-2025-08-07"
model = "gpt-4.1-mini"

## OpenAI Client
Azure OpenAI provides two methods for authentication. You can use either API Keys or Microsoft Entra ID.

* API Key authentication: For this type of authentication, all API requests must include the API Key in the api-key HTTP header. 

* Microsoft Entra ID authentication: You can authenticate an API call using a Microsoft Entra token. Authentication tokens are included in a request as the Authorization header. The token provided must be preceded by Bearer, for example Bearer YOUR_AUTH_TOKEN. You can read our how-to guide on authenticating with Microsoft Entra ID.

We will be using API Key for this class. Store the API Key in the `.env` file, never share your API Key with others. 


In [5]:
client = OpenAI(
    api_key=api_key,
    organization=org,
    project=project
)

In [6]:
[m.id for m in client.models.list().data if m.id.startswith("gpt-4.1")]

['gpt-4.1-2025-04-14',
 'gpt-4.1',
 'gpt-4.1-mini-2025-04-14',
 'gpt-4.1-mini',
 'gpt-4.1-nano-2025-04-14',
 'gpt-4.1-nano']

## Responses API 

In [10]:
response = client.responses.create(
    model=model,
    input="Hello, world!"
)

print(response.output_text)

Hello! How can I assist you today?


In [11]:
input=[
    {
        "role": "user",
        "content": [

                { "type": "input_text", "text": "describe " },
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                }
        ]
    }
]
response = client.responses.create(
    model=model,
    input=input
)


In [12]:
print(response.output_text)

The image shows a long wooden boardwalk cutting through a vast, lush green field. The grass on either side is tall and vibrant, with some scattered bushes and trees in the background. The sky above is wide and mostly clear, painted with a blue hue and streaks of wispy clouds. The scene is bright and peaceful, suggesting a pleasant day in a natural, open landscape.


## Responses API Streaming


In [46]:
response_stream = client.responses.create(
    model=model,
    input="write a long essage on a topic of your own!",
    stream=True
)


In [47]:
for event in response_stream:
    if event.type == 'response.output_text.delta':
        print(event.delta, end='')


Certainly! Here’s a long message on the topic of **“The Importance of Cultivating Empathy in Today’s World.”**

---

In an increasingly interconnected but often divided world, cultivating empathy has never been more essential. Empathy—the ability to understand and share the feelings of another—is a fundamental human trait that fosters compassion, strengthens relationships, and promotes social harmony. As societies rapidly evolve with technological advancements, cultural exchanges, and global challenges, the need for empathy becomes a critical pillar for building a more just and inclusive world.

One of the most powerful aspects of empathy is its ability to bridge differences. In many places, people are divided by race, religion, politics, socioeconomic status, or cultural norms. When empathy is practiced, individuals move beyond stereotypes and prejudices, seeking to understand the experiences and emotions of others. This understanding dismantles barriers, reduces conflict, and encoura

In [38]:
print(chunk.to_json())
print(chunk.item.status)

{
  "response": {
    "id": "resp_68c25833df14819698e2f8a090afac81061c92d8112a0e6d",
    "created_at": 1757567027.0,
    "error": null,
    "incomplete_details": null,
    "instructions": null,
    "metadata": {},
    "model": "gpt-4.1-mini-2025-04-14",
    "object": "response",
    "output": [
      {
        "id": "msg_68c25834c86881968f8013492b8b194c061c92d8112a0e6d",
        "content": [
          {
            "annotations": [],
            "text": "Hello! How can I assist you today?",
            "type": "output_text",
            "logprobs": []
          }
        ],
        "role": "assistant",
        "status": "completed",
        "type": "message"
      }
    ],
    "parallel_tool_calls": true,
    "temperature": 1.0,
    "tool_choice": "auto",
    "tools": [],
    "top_p": 1.0,
    "background": false,
    "max_output_tokens": null,
    "max_tool_calls": null,
    "previous_response_id": null,
    "prompt_cache_key": null,
    "reasoning": {
      "effort": null,
      "sum

AttributeError: 'ResponseCompletedEvent' object has no attribute 'item'

# Moderation models


Classifies if text and/or image inputs are potentially harmful. 

https://platform.openai.com/docs/guides/moderation

In [64]:
[m.id for m in client.models.list().data if m.id.__contains__("mode")]

['omni-moderation-latest', 'omni-moderation-2024-09-26']

In [65]:
moderation = client.moderations.create(input="I want to kill them.")
print(moderation.to_json())

{
  "id": "modr-CEPK9zRiUmu3n6D6My87Ab4M7PxRM",
  "model": "text-moderation-007",
  "results": [
    {
      "categories": {
        "harassment": true,
        "harassment/threatening": true,
        "hate": false,
        "hate/threatening": false,
        "self-harm": false,
        "self-harm/instructions": false,
        "self-harm/intent": false,
        "sexual": false,
        "sexual/minors": false,
        "violence": true,
        "violence/graphic": false,
        "self-harm": false,
        "sexual/minors": false,
        "hate/threatening": false,
        "violence/graphic": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "harassment/threatening": true
      },
      "category_scores": {
        "harassment": 0.5215635299682617,
        "harassment/threatening": 0.5694745779037476,
        "hate": 0.22706663608551025,
        "hate/threatening": 0.023547329008579254,
        "self-harm": 2.227119921371923e-6,
        "self-harm/i

In [73]:
#load image from base64 file
with open("image_base64_2.txt", "r") as f:
    image_base64 = f.read()

In [74]:
image_moderation = client.moderations.create(
    model="omni-moderation-latest",
    input=[
        # {"type": "text", "text": "...text to classify goes here..."},
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{image_base64}"
            }
        },
    ],

)


In [ ]:
moderation_response = image_moderation.to_json()
print(image_moderation.to_json())

{
  "id": "modr-5219",
  "model": "omni-moderation-latest",
  "results": [
    {
      "categories": {
        "harassment": false,
        "harassment/threatening": false,
        "hate": false,
        "hate/threatening": false,
        "illicit": false,
        "illicit/violent": false,
        "self-harm": false,
        "self-harm/instructions": false,
        "self-harm/intent": false,
        "sexual": true,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false,
        "harassment/threatening": false,
        "hate/threatening": false,
        "illicit/violent": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "self-harm": false,
        "sexual/minors": false,
        "violence/graphic": false
      },
      "category_applied_input_types": {
        "harassment": [],
        "harassment/threatening": [],
        "hate": [],
        "hate/threatening": [],
        "illicit": [],
        "illicit/vi

In [81]:
from types import SimpleNamespace


moderation_data = json.loads(moderation_response, object_hook=lambda d: SimpleNamespace(**d))

In [83]:
moderation_data.results

[namespace(categories=namespace(harassment=False,
                                harassment/threatening=False,
                                hate=False,
                                hate/threatening=False,
                                illicit=False,
                                illicit/violent=False,
                                self-harm=False,
                                self-harm/instructions=False,
                                self-harm/intent=False,
                                sexual=True,
                                sexual/minors=False,
                                violence=False,
                                violence/graphic=False),
           category_applied_input_types=namespace(harassment=[],
                                                  harassment/threatening=[],
                                                  hate=[],
                                                  hate/threatening=[],
                                            